In [ ]:
from sklearn.model_selection import train_test_split
import gensim
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, CategoricalNB, ComplementNB
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import hamming_loss

import ast

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

#Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_preprocess_nostemming.csv')
dataword2vec = df["review_content"]
df.head()

,Unnamed: 0,review_content,review_type,rev_clean,rev_token,rev_removesw
0,145,as a former middle school teacher of gifted li...,0,as a former middle school teacher of gifted li...,"['as', 'a', 'former', 'middle', 'school', 'tea...","['middle', 'school', 'teacher', 'gifted', 'lit..."
1,146,the lightning thief is an admirable kid's fant...,0,the lightning thief is an admirable kid s fant...,"['the', 'lightning', 'thief', 'is', 'an', 'adm...","['lightning', 'thief', 'admirable', 'kid', 'fa..."
2,147,"ok, but only just.",1,ok but only just,"['ok', 'but', 'only', 'just']",[]
3,148,there's nothing resembling a spark in this fil...,1,there s nothing resembling a spark in this fil...,"['there', 's', 'nothing', 'resembling', 'a', '...","['resembling', 'spark', 'film']"
4,289,please give is truly a film for our age; it's ...,0,please give is truly a film for our age it s d...,"['please', 'give', 'is', 'truly', 'a', 'film',...","['film', 'age', 'darkly', 'comic', 'stuff', 'm..."


In [ ]:
import seaborn as sns
import plotly.express as px
fig = px.bar(df.groupby('review_type').size().reset_index(name='count'), 
             x='review_type', y='count', width=480, height=480, 
             barmode='group', title='Distribusi label')
fig.show(renderer='colab')

In [ ]:
X = df[['review_content','rev_removesw']]
y = df['review_type']

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
cc = RandomUnderSampler()
X_resampled,y_resampled= cc.fit_resample(X,y)

In [ ]:
from collections import Counter

In [ ]:
print(sorted(Counter(y_resampled).items()))

[(0, 11757), (1, 11757)]


In [ ]:
unique, counts = np.unique(y_resampled, return_counts=True)

print(unique,counts)

[0 1] [11757 11757]


In [ ]:
df_resample = pd.DataFrame(X_resampled)
df_resample = df_resample.join(y_resampled)

In [ ]:
df_resample.head()

,review_content,rev_removesw,review_type
0,although [the movie] rightfully gives credit t...,"['movie', 'rightfully', 'credit', 'defense', '...",0
1,davis went big...debuting film actor jovan ade...,"['davis', 'big', 'debuting', 'film', 'actor', ...",0
2,mcqueen seems too enamored of his shorthand ar...,"['mcqueen', 'enamored', 'shorthand', 'art', 'h...",0
3,"the contrast between hudson's stolid, genteel ...","['contrast', 'hudson', 'stolid', 'genteel', 'a...",0
4,a simple concept yields a complex portrait of ...,"['simple', 'concept', 'yields', 'complex', 'po...",0


In [ ]:
import seaborn as sns
import plotly.express as px
fig = px.bar(df_resample.groupby('review_type').size().reset_index(name='count'), 
             x='review_type', y='count', width=480, height=480, 
             barmode='group', title='Distribusi label')
fig.show(renderer='colab')

In [ ]:
#df_resample.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_resample_nostem.csv')

In [ ]:
df = pd.read_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_resample_nostem.csv')
df.head()

,Unnamed: 0,review_content,rev_removesw,review_type
0,0,although [the movie] rightfully gives credit t...,"['movie', 'rightfully', 'credit', 'defense', '...",0
1,1,davis went big...debuting film actor jovan ade...,"['davis', 'big', 'debuting', 'film', 'actor', ...",0
2,2,mcqueen seems too enamored of his shorthand ar...,"['mcqueen', 'enamored', 'shorthand', 'art', 'h...",0
3,3,"the contrast between hudson's stolid, genteel ...","['contrast', 'hudson', 'stolid', 'genteel', 'a...",0
4,4,a simple concept yields a complex portrait of ...,"['simple', 'concept', 'yields', 'complex', 'po...",0


In [ ]:
print('Sentiment : ', df.groupby(['review_type'])['review_type'].count())

Sentiment :  review_type
0    11757
1    11757
Name: review_type, dtype: int64


In [ ]:
#masukkan kata ke dalam list soalnya kalo ga pake ini, dia datanya perkata untuk di pake data LDA
def convert_text_list(texts): 
    texts = ast.literal_eval(texts)
    return [text for text in texts]
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [ ]:
df["rev_list"] = df['rev_removesw'].apply(convert_text_list)
df["rev_kalimat"] = df["rev_list"].apply(listToString)

In [ ]:
df.head()

,Unnamed: 0,review_content,rev_removesw,review_type,rev_list,rev_kalimat
0,0,although [the movie] rightfully gives credit t...,"['movie', 'rightfully', 'credit', 'defense', '...",0,"[movie, rightfully, credit, defense, team, doc...",movie rightfully credit defense team documenta...
1,1,davis went big...debuting film actor jovan ade...,"['davis', 'big', 'debuting', 'film', 'actor', ...",0,"[davis, big, debuting, film, actor, jovan, ade...",davis big debuting film actor jovan adepo shines
2,2,mcqueen seems too enamored of his shorthand ar...,"['mcqueen', 'enamored', 'shorthand', 'art', 'h...",0,"[mcqueen, enamored, shorthand, art, house, sym...",mcqueen enamored shorthand art house symbolism...
3,3,"the contrast between hudson's stolid, genteel ...","['contrast', 'hudson', 'stolid', 'genteel', 'a...",0,"[contrast, hudson, stolid, genteel, acting, st...",contrast hudson stolid genteel acting style de...
4,4,a simple concept yields a complex portrait of ...,"['simple', 'concept', 'yields', 'complex', 'po...",0,"[simple, concept, yields, complex, portrait, c...",simple concept yields complex portrait contemp...


In [ ]:
df.isna().sum()

Unnamed: 0        0
review_content    3
rev_removesw      0
review_type       0
rev_list          0
rev_kalimat       0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum()

Unnamed: 0        0
review_content    0
rev_removesw      0
review_type       0
rev_list          0
rev_kalimat       0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23511 entries, 0 to 23513
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      23511 non-null  int64 
 1   review_content  23511 non-null  object
 2   rev_removesw    23511 non-null  object
 3   review_type     23511 non-null  int64 
 4   rev_list        23511 non-null  object
 5   rev_kalimat     23511 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.3+ MB


Split data

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

def splitData(df):
    X_train, X_test, y_train, y_test = train_test_split(df[['review_content','rev_list','rev_kalimat']], df['review_type'], test_size = 0.3, random_state= 42, shuffle = True)

    df_train = pd.DataFrame(y_train)  
    df_train = df_train.join(X_train)

    df_test = pd.DataFrame(y_test)
    df_test = df_test.join(X_test)

    return df_train, df_test

In [ ]:
data_train, data_test = splitData(df)

In [ ]:
data_train

,review_type,review_content,rev_list,rev_kalimat
14475,1,there is quite a bit to dark places that appea...,"[bit, dark, places, appears, marketable, surface]",bit dark places appears marketable surface
18185,1,the film tries to cover the logic gaps with co...,"[film, cover, logic, gaps, comedy, jokes, bad,...",film cover logic gaps comedy jokes bad worsen ...
3796,0,#unfit is a timely and vital documentary on th...,"[unfit, timely, vital, documentary, nature, pr...",unfit timely vital documentary nature presiden...
14280,1,what starts out as a promising comic thriller ...,"[starts, promising, comic, thriller, deflates,...",starts promising comic thriller deflates clear...
1281,0,i was pleasantly surprised by this movie.,"[pleasantly, surprised, movie]",pleasantly surprised movie
...,...,...,...,...
11965,1,the ten commandments may have been impressive ...,"[commandments, impressive, time, film, horribl...",commandments impressive time film horribly fai...
21578,1,"not as weird or fun as it sounds, the delibera...","[weird, fun, sounds, deliberately, paced, film...",weird fun sounds deliberately paced film dabbl...
5391,0,another beautifully rendered portrait of troub...,"[beautifully, rendered, portrait, troubled, ho...",beautifully rendered portrait troubled hopeful...
861,0,splendid pictorial composition and dramatic li...,"[splendid, pictorial, composition, dramatic, l...",splendid pictorial composition dramatic lighti...


In [ ]:
data_train.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_nostem.csv')

In [ ]:
data_test.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_nostem.csv')

In [ ]:
print('Jumlah Data :', len(data_train))
print('Sentiment : ', data_train.groupby(['review_type'])['review_type'].count())

Jumlah Data : 16457
Sentiment :  review_type
0    8234
1    8223
Name: review_type, dtype: int64


In [ ]:
print('Jumlah Data :', len(data_test))
print('Sentiment : ', data_test.groupby(['review_type'])['review_type'].count())

Jumlah Data : 7054
Sentiment :  review_type
0    3522
1    3532
Name: review_type, dtype: int64


#Word2Vec

##Train

In [ ]:
import pandas as pd
import numpy as np

# w2v
import joblib
import ast
import nltk
nltk.download('punkt')
import gensim
from gensim.models import Word2Vec 
import itertools
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
Corpus_voc = [] 
temp = []
# iterate through each sentence in the file 
for i in dataword2vec: 
  temp = [] 
  # tokenize the sentence into words 
  for j in word_tokenize(str(i)): 
      temp.append(j.lower()) 
  Corpus_voc.append(temp)

In [ ]:
data_train['rev_list'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         data_train.rev_list))  # tokenize sentences

In [ ]:
modelskip = Word2Vec(Corpus_voc, min_count = 1 , alpha = 0.1 , size=300, window=5, sg=1)

In [ ]:
joblib.dump(modelskip, '/content/drive/My Drive/TUGAS AKHIR/TA/Code/word2vecskip.jl')
skipgram = joblib.load('/content/drive/My Drive/TUGAS AKHIR/TA/Code/word2vecskip.jl')

In [ ]:
def get_w2v_features(w2v_model, sentence_group):
    words = np.concatenate(sentence_group)  # words in text
    index2word_set = set(w2v_model.wv.vocab.keys())  # words known to model
    
    featureVec = np.zeros(w2v_model.vector_size, dtype="float32")
    
    # Initialize a counter for number of words in a review
    nwords = 0
    # Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec = np.add(featureVec, w2v_model[word])
            nwords += 1.

    # Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

#mengambil arti dari semua vektor kata yang ada pada kalimat untuk mendapatkan representasi vektor 

In [ ]:
# memberi nomor index
index_kosong = []
for i in range(len(data_train.rev_list)):
  if len(data_train.rev_list.iloc[i]) == 0:
    index_kosong.append(i)

In [ ]:
# drop index yang masih kosong 
data_train = data_train.drop(data_train.index[index_kosong])

In [ ]:
data_train

,review_type,review_content,rev_list,rev_kalimat
14475,1,there is quite a bit to dark places that appea...,"[[bit], [dark], [places], [appears], [marketab...",bit dark places appears marketable surface
18185,1,the film tries to cover the logic gaps with co...,"[[film], [cover], [logic], [gaps], [comedy], [...",film cover logic gaps comedy jokes bad worsen ...
3796,0,#unfit is a timely and vital documentary on th...,"[[unfit], [timely], [vital], [documentary], [n...",unfit timely vital documentary nature presiden...
14280,1,what starts out as a promising comic thriller ...,"[[starts], [promising], [comic], [thriller], [...",starts promising comic thriller deflates clear...
1281,0,i was pleasantly surprised by this movie.,"[[pleasantly], [surprised], [movie]]",pleasantly surprised movie
...,...,...,...,...
11965,1,the ten commandments may have been impressive ...,"[[commandments], [impressive], [time], [film],...",commandments impressive time film horribly fai...
21578,1,"not as weird or fun as it sounds, the delibera...","[[weird], [fun], [sounds], [deliberately], [pa...",weird fun sounds deliberately paced film dabbl...
5391,0,another beautifully rendered portrait of troub...,"[[beautifully], [rendered], [portrait], [troub...",beautifully rendered portrait troubled hopeful...
861,0,splendid pictorial composition and dramatic li...,"[[splendid], [pictorial], [composition], [dram...",splendid pictorial composition dramatic lighti...


In [ ]:
data_train['w2v_features_skip'] = list(map(lambda sen_group:
                                      get_w2v_features(modelskip, sen_group),
                                      data_train.rev_list))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [ ]:
feature = [x for x in data_train['w2v_features_skip'].transpose()]
word2vec_with_stop_features_train = np.asarray(feature)

print(word2vec_with_stop_features_train.shape)

(16443, 300)


In [ ]:
X_train = word2vec_with_stop_features_train

In [ ]:
data_train

,review_type,review_content,rev_list,rev_kalimat,w2v_features_skip
14475,1,there is quite a bit to dark places that appea...,"[[bit], [dark], [places], [appears], [marketab...",bit dark places appears marketable surface,"[0.017600805, 0.045525413, -0.07878978, -0.157..."
18185,1,the film tries to cover the logic gaps with co...,"[[film], [cover], [logic], [gaps], [comedy], [...",film cover logic gaps comedy jokes bad worsen ...,"[0.03277949, -0.0046977606, -0.00041346115, -0..."
3796,0,#unfit is a timely and vital documentary on th...,"[[unfit], [timely], [vital], [documentary], [n...",unfit timely vital documentary nature presiden...,"[0.125936, 0.07559951, 0.0491084, -0.075301684..."
14280,1,what starts out as a promising comic thriller ...,"[[starts], [promising], [comic], [thriller], [...",starts promising comic thriller deflates clear...,"[0.05142096, 0.04879675, 0.023469325, -0.02128..."
1281,0,i was pleasantly surprised by this movie.,"[[pleasantly], [surprised], [movie]]",pleasantly surprised movie,"[-0.42745885, -0.019060785, -0.06350877, 0.053..."
...,...,...,...,...,...
11965,1,the ten commandments may have been impressive ...,"[[commandments], [impressive], [time], [film],...",commandments impressive time film horribly fai...,"[0.0014483721, 0.16194984, -0.0636947, -0.1311..."
21578,1,"not as weird or fun as it sounds, the delibera...","[[weird], [fun], [sounds], [deliberately], [pa...",weird fun sounds deliberately paced film dabbl...,"[-0.13050662, 0.123967, 0.008662553, -0.189898..."
5391,0,another beautifully rendered portrait of troub...,"[[beautifully], [rendered], [portrait], [troub...",beautifully rendered portrait troubled hopeful...,"[0.12951027, 0.14490424, -0.1077309, -0.177131..."
861,0,splendid pictorial composition and dramatic li...,"[[splendid], [pictorial], [composition], [dram...",splendid pictorial composition dramatic lighti...,"[-0.06982876, -0.014956219, 0.03542628, -0.273..."


##Test

In [ ]:
data_test['rev_list'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         data_test.rev_list))  # tokenize sentences

In [ ]:
# memberi nomor index
index_kosong = []
for i in range(len(data_test.rev_list)):
  if len(data_test.rev_list.iloc[i]) == 0:
    index_kosong.append(i)

In [ ]:
# drop index yang masih kosong 
data_test = data_test.drop(data_test.index[index_kosong])

In [ ]:
data_test['w2v_features_skip'] = list(map(lambda sen_group:
                                      get_w2v_features(modelskip, sen_group),
                                      data_test.rev_list))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [ ]:
feature = [x for x in data_test['w2v_features_skip'].transpose()]
word2vec_with_stop_features_test = np.asarray(feature)

print(word2vec_with_stop_features_test.shape)

(7048, 300)


In [ ]:
X_test = word2vec_with_stop_features_test

In [ ]:
data_test

,review_type,review_content,rev_list,rev_kalimat,w2v_features_skip
9660,0,the movie is full of wonderful character playe...,"[[movie], [wonderful], [character], [players],...",movie wonderful character players fun trumbo r...,"[0.04152465, 0.08956647, 0.045683637, -0.11204..."
15581,1,margot robbie and will smith share a tangible ...,"[[margot], [robbie], [smith], [share], [tangib...",margot robbie smith share tangible chemistry s...,"[-0.102406874, -0.11529929, 0.17369594, -0.243..."
12059,1,"committed isn't [heather] graham's fault. if, ...","[[committed], [heather], [graham], [fault], [t...",committed heather graham fault time screen cha...,"[0.07697252, 0.079500936, -0.05447163, -0.1047..."
8888,0,bad boys for life isn't only good; it may be t...,"[[bad], [boys], [life], [good], [series]]",bad boys life good series,"[0.13910103, 0.35064566, 0.02677458, -0.195291..."
9845,0,"flash of genius is a sincere and moving work, ...","[[flash], [genius], [moving], [work], [pedestr...",flash genius moving work pedestrian,"[0.045609806, 0.10004594, -0.032505434, -0.139..."
...,...,...,...,...,...
15028,1,"throughout his career as an actor, elvis presl...","[[career], [actor], [elvis], [presley], [invol...",career actor elvis presley involved cinematic ...,"[0.0071809185, 0.12341622, 0.052112084, -0.096..."
4246,0,that's a lot of serious talent behind this net...,"[[lot], [talent], [netflix], [original], [larg...",lot talent netflix original large pull jaiswal...,"[0.05227098, 0.039380293, -0.09392311, -0.0770..."
2311,0,[richard] attenborough manages to skillfully b...,"[[richard], [attenborough], [manages], [skillf...",richard attenborough manages skillfully balanc...,"[-0.12757997, -0.022111269, -0.044733576, -0.0..."
20458,1,[t]he decision to film disaster as a comedy re...,"[[decision], [film], [disaster], [comedy], [re...",decision film disaster comedy relegates charac...,"[-0.022652036, 0.0076340525, 0.1171721, -0.100..."


#NB

##Gaussian

In [ ]:
y_train = data_train['review_type']
y_test = data_test['review_type']

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)
pred = model.predict(X_train) #prediksi train
Y_pred = model.predict(X_test) #prediksi test
Y_pred


array([0, 0, 1, ..., 0, 1, 0])

### Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test, Y_pred)*100, "%")
print("Precision: ", precision_score(y_test, Y_pred)*100, "%")
print("Recall: ", recall_score(y_test, Y_pred)*100, "%")
print("F1 Score: ", f1_score(y_test, Y_pred)*100, "%")

Accuracy: 68.72871736662883 %
Precision:  71.40544041450777 %
Recall:  62.53545093590471 %
F1 Score:  66.67674629573631 %


In [ ]:
print("Confusion Matrix")
print(confusion_matrix(y_test, Y_pred))
tn, fp, fn, tp = confusion_matrix(y_test, Y_pred).ravel()
print('TN: ', tn)
print('FP: ', fp)
print('FN: ', fn)
print('TP: ', tp)

Confusion Matrix
[[2639  883]
 [1321 2205]]
TN:  2639
FP:  883
FN:  1321
TP:  2205


In [ ]:
print("Classification report")
print(classification_report(y_test, Y_pred))

Classification report
              precision    recall  f1-score   support

           0       0.67      0.75      0.71      3522
           1       0.71      0.63      0.67      3526

    accuracy                           0.69      7048
   macro avg       0.69      0.69      0.69      7048
weighted avg       0.69      0.69      0.69      7048



### Export data

In [ ]:
data_train_gaus = data_train.assign(sentiment_pred = pred)

In [ ]:
data_train_gaus.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_gaussian_300_pred_nostem.csv')

In [ ]:
data_test_gaus = data_test.assign(sentiment_pred = Y_pred)

In [ ]:
data_test_gaus.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_gaussian_300_pred_nostem.csv')

##Multinomial

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_multinomial = scaler.fit_transform(X_train)
X_test_multinomial = scaler.fit_transform(X_test)

In [ ]:
model_multinomial = MultinomialNB()
model_multinomial.fit(X_train_multinomial, y_train)
pred_multinomial = model_multinomial.predict(X_train_multinomial) #prediksi train
Y_pred_multinomial = model_multinomial.predict(X_test_multinomial) #prediksi test
Y_pred_multinomial

array([0, 0, 1, ..., 0, 1, 0])

### Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test, Y_pred_multinomial)*100, "%")
print("Precision: ", precision_score(y_test, Y_pred_multinomial)*100, "%")
print("Recall: ", recall_score(y_test, Y_pred_multinomial)*100, "%")
print("F1 Score: ", f1_score(y_test, Y_pred_multinomial)*100, "%")

Accuracy: 72.23325766174801 %
Precision:  71.06040268456375 %
Recall:  75.07090187180941 %
F1 Score:  73.01061922493449 %


In [ ]:
print("Confusion Matrix")
print(confusion_matrix(y_test, Y_pred_multinomial))
tn, fp, fn, tp = confusion_matrix(y_test, Y_pred_multinomial).ravel()
print('TN: ', tn)
print('FP: ', fp)
print('FN: ', fn)
print('TP: ', tp)

Confusion Matrix
[[2444 1078]
 [ 879 2647]]
TN:  2444
FP:  1078
FN:  879
TP:  2647


In [ ]:
print("Classification report")
print(classification_report(y_test, Y_pred_multinomial))

Classification report
              precision    recall  f1-score   support

           0       0.74      0.69      0.71      3522
           1       0.71      0.75      0.73      3526

    accuracy                           0.72      7048
   macro avg       0.72      0.72      0.72      7048
weighted avg       0.72      0.72      0.72      7048



### Export data

In [ ]:
data_train_ex_multi = data_train.assign(sentiment_pred = pred_multinomial)

In [ ]:
data_train_ex_multi.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_multinomial_300_pred_nostem.csv')

In [ ]:
data_test_ex_multi = data_test.assign(sentiment_pred = Y_pred_multinomial)

In [ ]:
data_test_ex_multi.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_multinomial_300_pred_nostem.csv')



#Dimensi 100

In [ ]:
data_train2 = pd.read_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_nostem.csv')
data_train2.head()

,review_type,review_content,rev_list,rev_kalimat
14475,1,there is quite a bit to dark places that appea...,"[bit, dark, places, appears, marketable, surface]",bit dark places appears marketable surface
18185,1,the film tries to cover the logic gaps with co...,"[film, cover, logic, gaps, comedy, jokes, bad,...",film cover logic gaps comedy jokes bad worsen ...
3796,0,#unfit is a timely and vital documentary on th...,"[unfit, timely, vital, documentary, nature, pr...",unfit timely vital documentary nature presiden...
14280,1,what starts out as a promising comic thriller ...,"[starts, promising, comic, thriller, deflates,...",starts promising comic thriller deflates clear...
1281,0,i was pleasantly surprised by this movie.,"[pleasantly, surprised, movie]",pleasantly surprised movie
...,...,...,...,...
11965,1,the ten commandments may have been impressive ...,"[commandments, impressive, time, film, horribl...",commandments impressive time film horribly fai...
21578,1,"not as weird or fun as it sounds, the delibera...","[weird, fun, sounds, deliberately, paced, film...",weird fun sounds deliberately paced film dabbl...
5391,0,another beautifully rendered portrait of troub...,"[beautifully, rendered, portrait, troubled, ho...",beautifully rendered portrait troubled hopeful...
861,0,splendid pictorial composition and dramatic li...,"[splendid, pictorial, composition, dramatic, l...",splendid pictorial composition dramatic lighti...


In [ ]:
print('Jumlah Data :', len(data_train2))
print('Sentiment : ', data_train2.groupby(['review_type'])['review_type'].count())

Jumlah Data : 16457
Sentiment :  review_type
0    8234
1    8223
Name: review_type, dtype: int64


In [ ]:
data_test2 = pd.read_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_nostem.csv')
data_test2.head()

In [ ]:
print('Jumlah Data :', len(data_test2))
print('Sentiment : ', data_test2.groupby(['review_type'])['review_type'].count())

Jumlah Data : 7054
Sentiment :  review_type
0    3522
1    3532
Name: review_type, dtype: int64


##W2V Train

In [ ]:
Corpus_voc = [] 
temp = []
# iterate through each sentence in the file 
for i in dataword2vec: 
  temp = [] 
  # tokenize the sentence into words 
  for j in word_tokenize(str(i)): 
      temp.append(j.lower()) 
  Corpus_voc.append(temp)

In [ ]:
data_train2['rev_list'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         data_train2.rev_list))  # tokenize sentences

In [ ]:
modelskip2 = Word2Vec(Corpus_voc, min_count = 1 , alpha = 0.1 , size=100, window=5, sg=1)

In [ ]:
joblib.dump(modelskip2, '/content/drive/My Drive/TUGAS AKHIR/TA/Code/word2vecskip.jl')
skipgram = joblib.load('/content/drive/My Drive/TUGAS AKHIR/TA/Code/word2vecskip.jl')

In [ ]:
def get_w2v_features(w2v_model, sentence_group):
    words = np.concatenate(sentence_group)  # words in text
    index2word_set = set(w2v_model.wv.vocab.keys())  # words known to model
    
    featureVec = np.zeros(w2v_model.vector_size, dtype="float32")
    
    # Initialize a counter for number of words in a review
    nwords = 0
    # Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec = np.add(featureVec, w2v_model[word])
            nwords += 1.

    # Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

#mengambil arti dari semua vektor kata yang ada pada kalimat untuk mendapatkan representasi vektor 

In [ ]:
# memberi nomor index
index_kosong = []
for i in range(len(data_train2.rev_list)):
  if len(data_train2.rev_list.iloc[i]) == 0:
    index_kosong.append(i)

In [ ]:
# drop index yang masih kosong 
data_train2 = data_train2.drop(data_train2.index[index_kosong])

In [ ]:
data_train2['w2v_features_skip'] = list(map(lambda sen_group:
                                      get_w2v_features(modelskip2, sen_group),
                                      data_train2.rev_list))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [ ]:
feature = [x for x in data_train2['w2v_features_skip'].transpose()]
word2vec_with_stop_features_train2 = np.asarray(feature)

print(word2vec_with_stop_features_train2.shape)

(16443, 100)


In [ ]:
X_train2 = word2vec_with_stop_features_train2

##W2V Test

In [ ]:
data_test2['rev_list'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         data_test2.rev_list))  # tokenize sentences

In [ ]:
# memberi nomor index
index_kosong = []
for i in range(len(data_test2.rev_list)):
  if len(data_test2.rev_list.iloc[i]) == 0:
    index_kosong.append(i)

In [ ]:
# drop index yang masih kosong 
data_test2 = data_test2.drop(data_test2.index[index_kosong])

In [ ]:
data_test2['w2v_features_skip'] = list(map(lambda sen_group:
                                      get_w2v_features(modelskip2, sen_group),
                                      data_test2.rev_list))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [ ]:
feature = [x for x in data_test2['w2v_features_skip'].transpose()]
word2vec_with_stop_features_test2 = np.asarray(feature)

print(word2vec_with_stop_features_test2.shape)

(7048, 100)


In [ ]:
X_test2 = word2vec_with_stop_features_test2

##Gaussian

In [ ]:
y_train2 = data_train2['review_type']
y_test2 = data_test2['review_type']

In [ ]:
model = GaussianNB()
model.fit(X_train2, y_train2)
pred2 = model.predict(X_train2) #prediksi train
Y_pred2 = model.predict(X_test2) #prediksi test
Y_pred2

array([0, 0, 1, ..., 0, 1, 0])

### Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test2, Y_pred2)*100, "%")
print("Precision: ", precision_score(y_test2, Y_pred2)*100, "%")
print("Recall: ", recall_score(y_test2, Y_pred2)*100, "%")
print("F1 Score: ", f1_score(y_test2, Y_pred2)*100, "%")

Accuracy: 70.64415437003404 %
Precision:  72.0024161884627 %
Recall:  67.61202495745887 %
F1 Score:  69.73818926429722 %


In [ ]:
print("Confusion Matrix")
print(confusion_matrix(y_test2, Y_pred2))
tn, fp, fn, tp = confusion_matrix(y_test2, Y_pred2).ravel()
print('TN: ', tn)
print('FP: ', fp)
print('FN: ', fn)
print('TP: ', tp)

Confusion Matrix
[[2595  927]
 [1142 2384]]
TN:  2595
FP:  927
FN:  1142
TP:  2384


In [ ]:
print("Classification report")
print(classification_report(y_test2, Y_pred2))

Classification report
              precision    recall  f1-score   support

           0       0.69      0.74      0.71      3522
           1       0.72      0.68      0.70      3526

    accuracy                           0.71      7048
   macro avg       0.71      0.71      0.71      7048
weighted avg       0.71      0.71      0.71      7048



### Export data

In [ ]:
data_train_ex2 = data_train2.assign(sentiment_pred = pred2)

In [ ]:
data_train_ex2.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_gaussian_100_pred_nostem.csv')

In [ ]:
data_test_ex2 = data_test2.assign(sentiment_pred = Y_pred2)

In [ ]:
data_test_ex2.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_gaussian_100_pred_nostem.csv')

##Multinomial

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train2_multi = scaler.fit_transform(X_train2)
X_test2_multi = scaler.fit_transform(X_test2)

In [ ]:
model2_multi = MultinomialNB()
model2_multi.fit(X_train2_multi, y_train2)
pred2_multi = model2_multi.predict(X_train2_multi) #prediksi train
Y_pred2_multi = model2_multi.predict(X_test2_multi) #prediksi test
Y_pred2_multi

array([1, 0, 1, ..., 0, 1, 0])

### Evaluation

In [ ]:
print("Accuracy:", accuracy_score(y_test2, Y_pred2_multi)*100, "%")
print("Precision: ", precision_score(y_test2, Y_pred2_multi)*100, "%")
print("Recall: ", recall_score(y_test2, Y_pred2_multi)*100, "%")
print("F1 Score: ", f1_score(y_test2, Y_pred2_multi)*100, "%")

Accuracy: 71.59477866061293 %
Precision:  69.21331316187594 %
Recall:  77.85025524673851 %
F1 Score:  73.27816337426589 %


In [ ]:
print("Confusion Matrix")
print(confusion_matrix(y_test2, Y_pred2_multi))
tn, fp, fn, tp = confusion_matrix(y_test2, Y_pred2_multi).ravel()
print('TN: ', tn)
print('FP: ', fp)
print('FN: ', fn)
print('TP: ', tp)

Confusion Matrix
[[2301 1221]
 [ 781 2745]]
TN:  2301
FP:  1221
FN:  781
TP:  2745


In [ ]:
print("Classification report")
print(classification_report(y_test2, Y_pred2_multi))

Classification report
              precision    recall  f1-score   support

           0       0.75      0.65      0.70      3522
           1       0.69      0.78      0.73      3526

    accuracy                           0.72      7048
   macro avg       0.72      0.72      0.71      7048
weighted avg       0.72      0.72      0.71      7048



### Export data

In [ ]:
data_train_ex2_multi = data_train2.assign(sentiment_pred = pred2_multi)

In [ ]:
data_train_ex2_multi.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_train_multinomial_100_pred_nostem.csv')

In [ ]:
data_test_ex2_multi = data_test2.assign(sentiment_pred = Y_pred2_multi)

In [ ]:
data_test_ex2_multi.to_csv('drive/My Drive/TUGAS AKHIR/DATASET_2/Final Dataset/data_test_multinomial_100_pred_nostem.csv')